# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

# Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [7]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1. What test is appropriate for this problem? Does CLT apply?

A permutation test(bootstrap) is good for this problem because we can test as many times as we can with random choiced data.

CLT applies because our data size is large enough(4870 > 30).

## 2.What are the null and alternate hypotheses?

H0: There is no difference between races on the callback rate.

Ha: There is a difference callback rate between different races. 

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [38]:
white = data[data.race=='w']['call']
black = data[data.race=='b']['call']

In [41]:
white_mean = np.mean(white)
black_mean = np.mean(black)
white_std = np.std(white)
black_std = np.std(black)
mean_diff = np.abs(white_mean - black_mean)
std_error= np.sqrt(white_std**2/len(white) + black_std**2/len(black))

In [45]:
#Frequentist
#under 95% confidence interval, the z-score is 1.96
margin_of_error = 1.96 * std_error
print('margin of error: ' + str(margin_of_error))
#confidence interval
confidence_interval = [mean_diff + margin_of_error, mean_diff - margin_of_error]
print('confidence interval: ' + str(confidence_interval))
#P-value
t,p = stats.ttest_ind(white,black)
print('P-value: ' + str(p))

margin of error: 0.015255284385449893
confidence interval: [0.04728813924051047, 0.016777570469610682]
P-value: 3.940802103128886e-05


In [51]:
# Bootstrap
permutation_replicates = np.empty(10000)

for i in range(len(permutation_replicates)):
    permutaion_sample = np.random.permutation(np.concatenate((white, black)))
    white_per_mean = np.mean(permutaion_sample[:len(white)])
    black_per_mean = np.mean(permutaion_sample[len(white):])
    permutation_replicates[i] = np.abs(white_per_mean - black_per_mean)
p = np.sum(permutation_replicates > mean_diff)/len(permutation_replicates)    
print('P-value: ' + str(p))

P-value: 0.0


## 4. Write a story describing the statistical significance in the context or the original problem.

In US job market, racial discrimination is still a crucial issue. The analysis of the job application dataset has statistically supported this statement. </p>  
Assuming that there is no racial bias on the chance of receiving callbacks, we ran bootstrap permutation test and frequentist test. However, p values from both tests are far below 0.05, which means the assumption is not statistically true. So we conclude that black and white groups do receive different callbacks when applying for jobs. 

 

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Although we have confirmed that races is critical in callbak success, we lack information to conclude that it is the most important one. 

There do have many other factors in the dataset that may affect callback rate, such as gender, education and year of experience. We do need to conduct more analysis to find more factors that make a difference and I think multi-variable analysis will be helpful in investigating callbacks at a large scope.